In [3]:
# %pip install exa-py cerebras-cloud-sdk

from exa_py import Exa
from cerebras.cloud.sdk import Cerebras
from dotenv import load_dotenv
import os

load_dotenv()
# Add your API keys here
EXA_API_KEY = os.getenv("EXA_API_KEY")
CEREBRAS_API_KEY = os.getenv("CERERAS_API_KEY")

client = Cerebras(api_key = CEREBRAS_API_KEY)
exa = Exa(api_key = EXA_API_KEY)

print("✅ Setup complete")

✅ Setup complete


In [4]:
def search_web(query,num=5):
    res = exa.search_and_contents(
        query,
        type = "auto",
        num_results= num,
        text={"max_characters":1000}
    )
    return res.results

print("✅ search func ready")

✅ search func ready


In [5]:
def ask_ai(prompt):
    """Get AI response from Cerebras"""
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="llama-4-scout-17b-16e-instruct",
      max_tokens = 600,
      temperature = 0.2
    )
    return chat_completion.choices[0].message.content

print("✅ AI function ready")

✅ AI function ready


In [ ]:
def research_topic(query):
    """Main research function that orchestrates the entire process"""
    print(f"🔍 Researching: {query}")

    # Search for sources
    results = search_web(query, 5)
    print(f"📊 Found {len(results)} sources")

    # Get content from sources
    sources = []
    for result in results:
        content = result.text
        title = result.title
        if content and len(content) > 200:
            sources.append({
                "title": title,
                "content": content
            })

    print(f"📄 Scraped {len(sources)} sources")

    if not sources:
        return {"summary": "No sources found", "insights": []}

    # Create context for AI analysis
    context = f"Research query: {query}\n\nSources:\n"
    for i, source in enumerate(sources[:4], 1):
        context += f"{i}. {source['title']}: {source['content'][:400]}...\n\n"
        # ^^ get rid of this to use API params!
        # best practices - https://www.anthropic.com/engineering/built-multi-agent-research-system

    # Ask AI to analyze and synthesize
    prompt = f"""{context}

    Based on these sources, provide:
    1. A comprehensive summary (2-3 sentences)
    2. Three key insights as bullet points

    Format your response exactly like this:
    SUMMARY: [your summary here]

    INSIGHTS:
    - [insight 1]
    - [insight 2]
    - [insight 3]"""

    response = ask_ai(prompt)
    print("🧠 Analysis complete")

    return {"query": query, "sources": len(sources), "response": response}

print("✅ Research function ready")

✅ Research function ready


In [7]:
def deeper_research_topic(query):
    """Two-layer research for better depth"""
    print(f"🔍 Researching: {query}")

    # Layer 1: Initial search
    results = search_web(query, 6)
    sources = []
    for result in results:
        if result.text and len(result.text) > 200:
            sources.append({"title": result.title, "content": result.text})

    print(f"Layer 1: Found {len(sources)} sources")

    if not sources:
        return {"summary": "No sources found", "insights": []}

    # Get initial analysis and identify follow-up topic
    context1 = f"Research query: {query}\n\nSources:\n"
    for i, source in enumerate(sources[:4], 1):
        context1 += f"{i}. {source['title']}: {source['content'][:300]}...\n\n"

    follow_up_prompt = f"""{context1}

Based on these sources, what's the most important follow-up question that would deepen our understanding of "{query}"?

Respond with just a specific search query (no explanation):"""

    follow_up_query = ask_ai(follow_up_prompt).strip().strip('"')

    # Layer 2: Follow-up search
    print(f"Layer 2: Investigating '{follow_up_query}'")
    follow_results = search_web(follow_up_query, 4)

    for result in follow_results:
        if result.text and len(result.text) > 200:
            sources.append({"title": f"[Follow-up] {result.title}", "content": result.text})

    print(f"Total sources: {len(sources)}")

    # Final synthesis
    all_context = f"Research query: {query}\nFollow-up: {follow_up_query}\n\nAll Sources:\n"
    for i, source in enumerate(sources[:7], 1):
        all_context += f"{i}. {source['title']}: {source['content'][:300]}...\n\n"

    final_prompt = f"""{all_context}

Provide a comprehensive analysis:

SUMMARY: [3-4 sentences covering key findings from both research layers]

INSIGHTS:
- [insight 1]
- [insight 2]
- [insight 3]
- [insight 4]

DEPTH GAINED: [1 sentence on how the follow-up search enhanced understanding]"""

    response = ask_ai(final_prompt)
    return {"query": query, "sources": len(sources), "response": response}

print("✅ Enhanced research function ready")

✅ Enhanced research function ready


In [9]:
# Test the enhanced research system
result = deeper_research_topic("top trending AI innovation for the month of september, include all breakthroughs")

# Display results
print("\n" + "="*50)
print("ENHANCED RESEARCH RESULTS")
print("="*50)
print(f"Query: {result['query']}")
print(f"Sources analyzed: {result['sources']}")
print(f"\n{result['response']}")
print("="*50)

# Try more topics
print("\nTry these:")
print("deeper_research_topic('quantum computing advances')")
print("deeper_research_topic('space exploration news')")

🔍 Researching: top trending AI innovation for the month of september, include all breakthroughs
Layer 1: Found 6 sources
Layer 2: Investigating 'What are the specific applications and potential impact of AI-designed viruses, and how do they relate to the other breakthroughs in AI innovation for September 2025, such as enhanced planning capabilities and AI-generated materials with exotic quantum properties?'
Total sources: 10

ENHANCED RESEARCH RESULTS
Query: top trending AI innovation for the month of september, include all breakthroughs
Sources analyzed: 10

**SUMMARY**

The top trending AI innovations for September 2025 include breakthroughs in AI-designed viruses, enhanced planning capabilities, and AI-generated materials with exotic quantum properties. AI-designed viruses have shown promise in killing bacteria, while MIT researchers have developed an AI model that achieves 94% accuracy in planning tasks, a 64x improvement over previous models. Additionally, a new tool has been crea

In [10]:
def anthropic_multiagent_research(query):
    """
    Simple implementation of Anthropic's multi-agent approach:
    1. Lead agent plans and delegates
    2. Specialized subagents work in parallel
    3. Lead agent synthesizes results
    """
    print(f"🤖 Anthropic Multi-Agent Research: {query}")
    print("-" * 50)

    # Step 1: Lead Agent - Task Decomposition & Delegation
    print("👨‍💼 LEAD AGENT: Planning and delegating...")

    delegation_prompt = f"""You are a Lead Research Agent. Break down this complex query into 3 specialized subtasks for parallel execution: "{query}"

For each subtask, provide:
- Clear objective
- Specific search focus
- Expected output

SUBTASK 1: [Core/foundational aspects]
SUBTASK 2: [Recent developments/trends]
SUBTASK 3: [Applications/implications]

Make each subtask distinct to avoid overlap."""

    plan = ask_ai(delegation_prompt)
    print("  ✓ Subtasks defined and delegated")

    # Step 2: Simulate Parallel Subagents (simplified for demo)
    print("\n🔍 SUBAGENTS: Working in parallel...")

    # Extract subtasks and create targeted searches
    subtask_searches = [
        f"{query} fundamentals principles",  # Core aspects
        f"{query} latest developments",  # Recent trends
        f"{query} applications real world"    # Implementation
    ]

    subagent_results = []
    for i, search_term in enumerate(subtask_searches, 1):
        print(f"  🤖 Subagent {i}: Researching {search_term}")
        results = search_web(search_term, 2)

        sources = []
        for result in results:
            if result.text and len(result.text) > 200:
                sources.append({
                    "title": result.title,
                    "content": result.text[:300]
                })

        subagent_results.append({
            "subtask": i,
            "search_focus": search_term,
            "sources": sources
        })

    total_sources = sum(len(r["sources"]) for r in subagent_results)
    print(f"  📊 Combined: {total_sources} sources from {len(subagent_results)} agents")

    # Step 3: Lead Agent - Synthesis
    print("\n👨‍💼 LEAD AGENT: Synthesizing parallel findings...")

    # Combine all subagent findings
    synthesis_context = f"ORIGINAL QUERY: {query}\n\nSUBAGENT FINDINGS:\n"
    for result in subagent_results:
        synthesis_context += f"\nSubagent {result['subtask']} ({result['search_focus']}):\n"
        for source in result['sources'][:2]:  # Limit for brevity
            synthesis_context += f"- {source['title']}: {source['content']}...\n"

    synthesis_prompt = f"""{synthesis_context}

As the Lead Agent, synthesize these parallel findings into a comprehensive report:

EXECUTIVE SUMMARY:
[2-3 sentences covering the most important insights across all subagents]

INTEGRATED FINDINGS:
• [Key finding from foundational research]
• [Key finding from recent developments]
• [Key finding from applications research]
• [Cross-cutting insight that emerged]

RESEARCH QUALITY:
- Sources analyzed: {total_sources} across {len(subagent_results)} specialized agents
- Coverage: [How well the subtasks covered the topic]"""

    final_synthesis = ask_ai(synthesis_prompt)

    print("\n" + "=" * 50)
    print("🎯 MULTI-AGENT RESEARCH COMPLETE")
    print("=" * 50)
    print(final_synthesis)

    return {
        "query": query,
        "subagents": len(subagent_results),
        "total_sources": total_sources,
        "synthesis": final_synthesis
    }

print("✅ Anthropic multi-agent system ready!")
     

✅ Anthropic multi-agent system ready!


In [12]:

# Test the Anthropic multi-agent research system
result = anthropic_multiagent_research("quantum computing commercial applications")

print("\n" + "🤖" * 30)
print("ANTHROPIC MULTI-AGENT DEMO")
print("🤖" * 30)
print(f"Query: {result['query']}")
print(f"Subagents deployed: {result['subagents']}")
print(f"Total sources: {result['total_sources']}")
print("\n💡 Key Innovation: Parallel specialized agents + intelligent orchestration")

🤖 Anthropic Multi-Agent Research: quantum computing commercial applications
--------------------------------------------------
👨‍💼 LEAD AGENT: Planning and delegating...
  ✓ Subtasks defined and delegated

🔍 SUBAGENTS: Working in parallel...
  🤖 Subagent 1: Researching quantum computing commercial applications fundamentals principles
  🤖 Subagent 2: Researching quantum computing commercial applications latest developments
  🤖 Subagent 3: Researching quantum computing commercial applications applications real world
  📊 Combined: 6 sources from 3 agents

👨‍💼 LEAD AGENT: Synthesizing parallel findings...

🎯 MULTI-AGENT RESEARCH COMPLETE
**EXECUTIVE SUMMARY:**
Quantum computing is rapidly evolving from a theoretical concept to a tangible technology with significant commercial potential. Recent developments, such as HSBC's demonstration of quantum-enabled algorithmic trading, showcase the practical applications of quantum computing in industries like finance. As the technology advances, bus